In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle
import matplotlib
import ipyparams

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import func_optimize as fo
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
import neurolib.dashboard.layout as layout
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

cmap_ = layout.getcolormap()
darkgrey, midgrey, lightgrey, color_bi_updown, color_LC, color_bi_uposc = layout.getcolors_rgba()
exc_red = cmap_(3)
inh_blue=cmap_(0)
green = cmap_(2)

<IPython.core.display.Javascript object>

not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

def rep_ind(c0, c1):
    abs_diff = np.abs(c0-c1)
    res = 0.
    for n in range(c0.shape[0]):
        for v in range(c0.shape[1]):
            res += sum(abs_diff[n,v,:]) * dt
            
    return res

In [3]:
points = np.array( [ [0.4, 0.35], [0.4, 0.4], [0.45, 0.4], [0.425, 0.45], [0.475, 0.45], [0.45, 0.5],
                    [0.5, 0.5], [0.45, 0.65], [0.5, 0.65], [0.55, 0.65] ])

# 10 points, two costs, 4 methods, N 

In [4]:
def create_dict(task, point, cost_params, init, init_vars, target, sigma_array, N_max_array, it_array):
    return dict( {'task':task, 'point':point, 'cost_params':cost_params, 'init':init, 'init_vars':init_vars,
            'target':target, 'sigma':sigma_array, 'N_array':N_max_array, 'it_array':it_array,
            'cost':np.zeros(( len(sigma_array), 4, 2, len(N_max_array), len(it_array) )),
            'rep_ind':np.zeros(( len(sigma_array), 4, 2, len(N_max_array), len(it_array) ))     } )

params_list = ['task', 'point', 'cost_params', 'init', 'init_vars', 'target', 'sigma', 'N_array',
               'cost', 'rep_ind']

In [5]:
dict_list = []
sigma_array = np.arange(0.01, 0.081, 0.02)
N_array = [2,10,30,50]
it_array = np.arange(0,25,6)

for p in points:
    dict_list.append(create_dict('DU', p, [None]*len(sigma_array), [0], None,
                [None]*len(sigma_array), sigma_array, N_array, it_array) )

In [6]:
for d in dict_list:
    print(d['target'])

[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]
[None, None, None, None]


In [7]:
dict_file = "DU_1E.pickle"

if False:
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()

if True:
    file = open(dict_file, "rb")
    dict_list = pickle.load(file)
    file.close()

In [8]:
dur_pre = 10
dur_post = 10
dur = 200.
dur_comp = 2000.
dt = aln.params.dt
T = int(1 + dur / dt)

prec_vars = [0,1]
trans_time = 0.8

n_pre = int(np.around(dur_pre/aln.params.dt,1))
n_post = int(np.around(dur_post/aln.params.dt,1))

tol = 1e-16
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

maxC = [5., -5., 0.18, 0.]
n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

noise_realizations_final = 10000
noise_realizations_target_compute = 1000
init_it = 3

In [9]:
# get init vars without noise
aln = ALNModel()
data.set_parameters(aln)
aln.params.duration = dur_comp

# initial without noise
control0 = aln.getZeroControl()

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
    
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    aln.params.duration = dur_comp
    
    if type(d['init_vars']) == type(None):
    
        control0[0,0,:500] = -5.

        aln.run(control=control0)

        init_state_vars = np.zeros(( len(state_vars) ))
        for j in range(len(state_vars)):
            if aln.state[state_vars[j]].size == 1:
                init_state_vars[j] = aln.state[state_vars[j]][0]
            else:
                init_state_vars[j] = aln.state[state_vars[j]][0,-1]

        initVars = init_state_vars.copy()
        d['init_vars'] = initVars
    
    for ind_s in range(len(d['sigma'])):
        
        if type(d['target'][ind_s]) != type(None):
            continue        
        
        s = d['sigma'][ind_s]
        aln.params.sigma_ou = s
        aln.params.duration = dur_comp
        control0[0,0,:500] = 5.
        
        stateswitchcount = 0
        target_rates_e = []
        target_rates_i = []

        for j in range(noise_realizations_target_compute):
            if j % 100 == 0:
                print(j)
            aln.run(control=control0)
            #plt.plot(aln.t, aln.rates_exc[0,:])
            if np.amin(aln.rates_exc[0,1000:]) < 1.:
                #print("fell in down state, repeat")
                j -= 1
                stateswitchcount += 1
                if stateswitchcount > 0.8 * noise_realizations_target_compute:
                    print("UNSTABLE: point, sigma ", d['point'], s)
                    break
                continue
            target_rates_e.append(aln.rates_exc[0,1000:])
            target_rates_i.append(aln.rates_inh[0,1000:])
        
        #plt.show()            

        aln.params.duration = dur
        target = aln.getZeroTarget()
        target[:,0,:] = np.mean(target_rates_e)
        target[:,1,:] = np.mean(target_rates_i)
        stddev = np.std(target_rates_e) + np.std(target_rates_i)
        
        d['target'][ind_s] = target
        d['cost_params'][ind_s] = [1. / ( stddev * (1. - trans_time) * dur ), 0., 1.]
        
        #print(stddev)
        
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()
    
    break

Point =  [0.5 0.5]


In [36]:
for d in dict_list:
    for s_ind in range(len(d['sigma'])):
        for i_it in range(1,len(d['it_array'])):
            for n_ind in range(len(d['N_array'])):
                #d['cost'][s_ind, 0, 0, n_ind, i_it] = 0.

In [17]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 1
method_ind = 0
step_method_ind = 0

for d in dict_list:
    
    if d['point'][0] not in [0.4, 0.5]:
        continue
    if d['point'][1] not in [0.35, 0.5]:
        continue
        
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        aln.params.duration = dur
        zero_init = aln.getZeroControl()
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
        
        if np.amin(d['cost'][s_ind, method_ind, step_method_ind, :, 0]) == 0.:
        
            setinit(d['init_vars'], aln)
            cost_final = 0.

            for n_ in range(noise_realizations_final):
                state_ = fo.updateState(aln, zero_init)
                cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                               state_, target_, list(prec_vars) )

            d['cost'][s_ind, :, :, :, 0] = cost_final/noise_realizations_final
            
        print('cost = ', d['cost'][s_ind, 0, 0, 0])    
        
        for i_it in range(1,len(d['it_array'])):
            
            it = d['it_array'][i_it]
            print("iteration = ", it)
        
            for n_ind in range(len(d['N_array'])):

                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                    
                n_max = d['N_array'][n_ind]
                print("n max = ", n_max)

                aln.params.duration = dur
                     
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    control_init = aln.getZeroControl()               
                    setinit(d['init_vars'], aln)
                    
                    c_vars = d['init']

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_init = bc[:,:,n_pre:-n_post]
                    c_vars = [0,1]
                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)
                    
                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = it-init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init = control_sum / n_max
        
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )
                    cost_final += cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2],
                                                                   dt, control_init)

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = cost_final/noise_realizations_final
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                ###############################################################
                
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                     
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    control_init1 = aln.getZeroControl()               
                    setinit(d['init_vars'], aln)
                    
                    c_vars = d['init']

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_init1 = bc[:,:,n_pre:-n_post]
                    c_vars = [0,1]
                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)
                    
                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = it-init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init1 = control_sum / n_max
                
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()

Point =  [0.4  0.35]
sigma =  0.01
set cost params:  0.020159116793190712 0.0 1.0
cost =  [1620.12286954    6.59301926    7.29276913    7.01566087    5.59643328]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.006622217419901204 0.0 1.0
cost =  [525.19289811   6.81674846   7.07449391   5.01217897   5.0576799 ]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.049999999999999996
set cost params:  0.003809411513721921 0.0 1.0
cost =  [289.81198041   6.07605961  11.26446292  12.06454788   7.34344626]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.06999999999999999
set cost params:  0.0026559867977029032 0.0 1.0
cost =  [174.83921099   6.70608997   8.0266208    6.53659892   8.47110663]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
cost =  [3692.09189037    7.72495331    8.29880844    7.05414039  

RUN  2 , total integrated cost =  7.660273919773169
RUN  3 , total integrated cost =  7.458366134030266
RUN  3 , total integrated cost =  7.458366134030266
Improved over  3  iterations in  0.28201150498352945  seconds by  26.504429093933894  percent.
-------- 8
interpolate adjoint :  True True True
initial cost =  3688.3122108045673
RUN  0 , total integrated cost =  3688.3122108045673
Gradient descend method:  None
RUN  1 , total integrated cost =  140.43144664475807
RUN  2 , total integrated cost =  73.90435296623262
RUN  3 , total integrated cost =  60.39733530260534
RUN  3 , total integrated cost =  60.39733530260534
Improved over  3  iterations in  0.12025600101333112  seconds by  98.36246684525032  percent.
interpolate adjoint :  True True True
initial cost =  63.65025966096172
RUN  0 , total integrated cost =  63.65025966096172
Gradient descend method:  None
RUN  1 , total integrated cost =  14.32629489531773
RUN  2 , total integrated cost =  9.60055873483719
RUN  3 , total integ

RUN  3 , total integrated cost =  11.746757052858774
RUN  3 , total integrated cost =  11.746757052858774
Improved over  3  iterations in  0.26367638597730547  seconds by  83.79650542638379  percent.
-------- 17
interpolate adjoint :  True True True
initial cost =  3684.4129251251147
RUN  0 , total integrated cost =  3684.4129251251147
Gradient descend method:  None
RUN  1 , total integrated cost =  163.30726738125975
RUN  2 , total integrated cost =  57.21948443216863
RUN  3 , total integrated cost =  29.889477262156838
RUN  3 , total integrated cost =  29.889477262156838
Improved over  3  iterations in  0.14689665101468563  seconds by  99.18875875561255  percent.
interpolate adjoint :  True True True
initial cost =  30.75151519911141
RUN  0 , total integrated cost =  30.75151519911141
Gradient descend method:  None
RUN  1 , total integrated cost =  16.87273963537344
RUN  2 , total integrated cost =  6.255720784905018
RUN  3 , total integrated cost =  5.525319431809327
RUN  3 , total 

RUN  1 , total integrated cost =  7.944322214471391
RUN  2 , total integrated cost =  6.9050216766952905
RUN  3 , total integrated cost =  6.36959829510451
RUN  3 , total integrated cost =  6.36959829510451
Improved over  3  iterations in  0.2328032919904217  seconds by  79.38252250473496  percent.
-------- 26
interpolate adjoint :  True True True
initial cost =  3677.6484419338312
RUN  0 , total integrated cost =  3677.6484419338312
Gradient descend method:  None
RUN  1 , total integrated cost =  102.06766010534993
RUN  2 , total integrated cost =  39.73328159342696
RUN  3 , total integrated cost =  11.911633061699177
RUN  3 , total integrated cost =  11.911633061699177
Improved over  3  iterations in  0.13163077307399362  seconds by  99.67610734822071  percent.
interpolate adjoint :  True True True
initial cost =  12.312584089406116
RUN  0 , total integrated cost =  12.312584089406116
Gradient descend method:  None
RUN  1 , total integrated cost =  10.266508347685653
RUN  2 , total i

RUN  1 , total integrated cost =  10.349780519115182
RUN  2 , total integrated cost =  10.19412990487325
RUN  3 , total integrated cost =  10.087197294075166
RUN  3 , total integrated cost =  10.087197294075166
Improved over  3  iterations in  0.25730005698278546  seconds by  24.107401240079724  percent.
-------- 35
interpolate adjoint :  True True True
initial cost =  3675.5601488786433
RUN  0 , total integrated cost =  3675.5601488786433
Gradient descend method:  None
RUN  1 , total integrated cost =  150.44365690122194
RUN  2 , total integrated cost =  26.01062353563995
RUN  3 , total integrated cost =  6.74763355043101
RUN  3 , total integrated cost =  6.74763355043101
Improved over  3  iterations in  0.14399585698265582  seconds by  99.8164189054969  percent.
interpolate adjoint :  True True True
initial cost =  7.5408340941736025
RUN  0 , total integrated cost =  7.5408340941736025
Gradient descend method:  None
RUN  1 , total integrated cost =  5.608400918736702
RUN  2 , total i

...

RUN  14 , total integrated cost =  6.498196060696685
RUN  15 , total integrated cost =  6.407510472382522
RUN  15 , total integrated cost =  6.407510472382522
Improved over  15  iterations in  0.9566437670728192  seconds by  80.40034607190056  percent.
-------- 23
interpolate adjoint :  True True True
initial cost =  3646.537265803266
RUN  0 , total integrated cost =  3646.537265803266
Gradient descend method:  None
RUN  1 , total integrated cost =  103.94361119074425
RUN  2 , total integrated cost =  85.72877460751961
RUN  3 , total integrated cost =  44.610757854535464
RUN  3 , total integrated cost =  44.610757854535464
Improved over  3  iterations in  0.11621092504356056  seconds by  98.77662684890433  percent.
interpolate adjoint :  True True True
initial cost =  48.12829515480491
RUN  0 , total integrated cost =  48.12829515480491
Gradient descend method:  None
RUN  1 , total integrated cost =  14.054963522140696
RUN  2 , total integrated cost =  11.367437678186455
RUN  3 , total

RUN  1 , total integrated cost =  8.830724981658818
RUN  2 , total integrated cost =  7.094345288091915
RUN  3 , total integrated cost =  6.9422117250669855
RUN  4 , total integrated cost =  6.8688211317288275
RUN  5 , total integrated cost =  6.822983482512347
RUN  6 , total integrated cost =  6.724910436019139
RUN  7 , total integrated cost =  6.706917375247798
RUN  8 , total integrated cost =  6.682376669879933
RUN  9 , total integrated cost =  6.689250559382409
RUN  10 , total integrated cost =  6.635663814242722
RUN  11 , total integrated cost =  6.636638628485396
RUN  12 , total integrated cost =  6.632156929151028
RUN  13 , total integrated cost =  6.677681641690502
RUN  14 , total integrated cost =  6.619640299794693
RUN  15 , total integrated cost =  6.6104620199131805
RUN  15 , total integrated cost =  6.6104620199131805
Improved over  15  iterations in  1.093178741983138  seconds by  79.24531283005646  percent.
-------- 29
interpolate adjoint :  True True True
initial cost =

RUN  11 , total integrated cost =  22.13895044189091
RUN  12 , total integrated cost =  22.131095044872477
RUN  13 , total integrated cost =  22.083152224231995
RUN  14 , total integrated cost =  22.064302101577336
RUN  15 , total integrated cost =  22.14838153884246
RUN  15 , total integrated cost =  22.14838153884246
Improved over  15  iterations in  1.0200233380310237  seconds by  17.644532284982205  percent.
-------- 34
interpolate adjoint :  True True True
initial cost =  3695.769152908631
RUN  0 , total integrated cost =  3695.769152908631
Gradient descend method:  None
RUN  1 , total integrated cost =  155.4792190020322
RUN  2 , total integrated cost =  12.120201557035461
RUN  3 , total integrated cost =  7.810570021468665
RUN  3 , total integrated cost =  7.810570021468665
Improved over  3  iterations in  0.1275564730167389  seconds by  99.78866185364089  percent.
interpolate adjoint :  True True True
initial cost =  9.485985856227163
RUN  0 , total integrated cost =  9.4859858

RUN  2 , total integrated cost =  4.891324521796664
RUN  3 , total integrated cost =  4.584207709664799
RUN  4 , total integrated cost =  4.388727999927028
RUN  5 , total integrated cost =  4.453035718347605
RUN  6 , total integrated cost =  4.508711866724061
RUN  7 , total integrated cost =  4.366910886533414
RUN  8 , total integrated cost =  4.264255507948995
RUN  9 , total integrated cost =  4.307449460889057
RUN  10 , total integrated cost =  4.227251422661281
RUN  11 , total integrated cost =  4.188064116379099
RUN  12 , total integrated cost =  4.144366066719861
RUN  13 , total integrated cost =  4.12026818232056
RUN  14 , total integrated cost =  4.159138553444773
RUN  15 , total integrated cost =  4.135495110000545
RUN  15 , total integrated cost =  4.135495110000545
Improved over  15  iterations in  1.2460785959847271  seconds by  46.948334648876724  percent.
-------- 40
interpolate adjoint :  True True True
initial cost =  3679.844230767495
RUN  0 , total integrated cost =  3

RUN  13 , total integrated cost =  6.49817751282078
RUN  14 , total integrated cost =  6.470093874694562
RUN  15 , total integrated cost =  6.3799980831670435
RUN  15 , total integrated cost =  6.3799980831670435
Improved over  15  iterations in  1.0141006039921194  seconds by  78.84591456656302  percent.
-------- 45
interpolate adjoint :  True True True
initial cost =  3661.339965721976
RUN  0 , total integrated cost =  3661.339965721976
Gradient descend method:  None
RUN  1 , total integrated cost =  160.2310674001497
RUN  2 , total integrated cost =  116.13321707754199
RUN  3 , total integrated cost =  21.778239662323653
RUN  3 , total integrated cost =  21.778239662323653
Improved over  3  iterations in  0.11323037499096245  seconds by  99.40518389807517  percent.
interpolate adjoint :  True True True
initial cost =  21.705321620488917
RUN  0 , total integrated cost =  21.705321620488917
Gradient descend method:  None
RUN  1 , total integrated cost =  6.820203357639976
RUN  2 , tot

In [21]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 2
method_ind = 1
step_method_ind = 0

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3684.031047163612
Gradient descend method:  None
RUN  1 , total integrated cost =  157.09574190890905
RUN  2 , total integrated cost =  33.301617297873825
RUN  3 , total integrated cost =  7.916348218967609
RUN  3 , total integrated cost mean =  7.916348218967609
Improved over  3  iterations in  9.6430786139681  seconds by  99.78511722301954  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  7.581444871688859
Gradient descend method:  None
step size too small or too large
step size too small or too large
RUN  1 , total integrate

RUN  1 , total integrated cost =  8.507373968745235
RUN  2 , total integrated cost =  7.263509391053621
RUN  3 , total integrated cost =  4.621990630434575
RUN  4 , total integrated cost =  4.434072468020153
RUN  5 , total integrated cost =  5.5121558420283945
step size too small or too large
RUN  6 , total integrated cost =  4.124629986973841
RUN  7 , total integrated cost =  4.39635947598344
RUN  8 , total integrated cost =  4.559777241790084
RUN  9 , total integrated cost =  4.543211869374958
RUN  9 , total integrated cost mean =  4.543211869374958
Improved over  9  iterations in  2.1919245300232433  seconds by  73.74449211248648  percent.
rep_ind =  19.054417184722954
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1223.0240426676482
Gradient descend method:  None
RUN  1 , total integrated cost =  117.23082943929883
RUN  2 , total integrated cost =  26.988882981653376
RUN  3 , total integrated cost =  8.554014231103759
RUN  3 

RUN  18 , total integrated cost =  7.8372538360163855
RUN  19 , total integrated cost =  7.851919958656309
RUN  20 , total integrated cost =  7.789243546007131
RUN  21 , total integrated cost mean =  7.686532428865999
Improved over  21  iterations in  22.845268054981716  seconds by  23.778446223925854  percent.
cost =  7.940797476023635
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1224.3916168542103
Gradient descend method:  None
RUN  1 , total integrated cost =  52.04404418440301
RUN  2 , total integrated cost =  25.30895565906036
RUN  3 , total integrated cost =  5.942701501460452
RUN  3 , total integrated cost mean =  5.942701501460452
Improved over  3  iterations in  1.6796986749977805  seconds by  99.5146404614621  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  5.520653440047587
Gradient descend method:  None
RUN  1 , total integrated cost =  5.055187849469001
RUN  2 , total integrated cost =  4.742903101838152
step size

RUN  1 , total integrated cost =  8.241575514673226
step size too small or too large
RUN  2 , total integrated cost =  7.7690591387373376
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  7.669604452553543
step size too small or too large
RUN  4 , total integrated cost =  7.574950846083044
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  7.4078075196682365
RUN  6 , total integrated cost =  7.4546280503179085
RUN  7 , total integrated cost =  7.413044586360578
RUN  8 , total integrated cost =  7.302457972445844
RUN  9 , total integrated cost =  7.246746870139607
step size too small or too large
step size too small or too large
RUN  10 , total integrated cost =  7.216938843781807
step size too 

RUN  12 , total integrated cost =  5.761942914770508
step size too small or too large
step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  5.6676346852280775
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  14 , total integrated cost =  5.718104213627629
RUN  15 , total integrated cost =  5.696526434423345
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  16 , total integrated cost =  5.656422486128196
step size too small or too large
step size too small or too large
RUN  17 , total integrated cost =  5.634539832458589
step size too small or too large
step size too small or too large
RUN  18 , total integrated cost =  5.6087283577035825
step size too small or too large
step size too small or too large
RUN  19 , total integrated cost =

...

RUN  15 , total integrated cost =  6.528941032733374
step size too small or too large
RUN  16 , total integrated cost =  6.504379014463499
RUN  17 , total integrated cost =  6.436829064152851
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  18 , total integrated cost =  6.443750683101863
step size too small or too large
step size too small or too large
RUN  19 , total integrated cost =  6.412462739807521
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  20 , total integrated cost =  6.322532111394298
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or

RUN  1 , total integrated cost =  5.458240774535332
RUN  2 , total integrated cost =  5.6881007033671995
RUN  3 , total integrated cost =  5.5968949742690315
RUN  4 , total integrated cost =  6.241857671475336
RUN  5 , total integrated cost =  5.505874925466603
RUN  6 , total integrated cost =  5.363446483937359
RUN  7 , total integrated cost =  5.695323998976933
RUN  8 , total integrated cost =  6.051659277083104
no cost improvement
RUN  9 , total integrated cost =  5.845420093405155
RUN  10 , total integrated cost =  5.586330473717025
RUN  11 , total integrated cost =  5.716380696244135
RUN  12 , total integrated cost =  4.9982483146621846
RUN  12 , total integrated cost mean =  4.9982483146621846
Improved over  12  iterations in  2.5695312680327334  seconds by  18.521384398334405  percent.
cost =  6.04968314956386
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  500.50052502009675
Gradient descend method:  None
RUN  1 , total integrated cost =  22.599771003283

RUN  6 , total integrated cost =  6.0957517140974
RUN  7 , total integrated cost =  6.163516086184972
RUN  8 , total integrated cost =  6.014302801368585
step size too small or too large
RUN  9 , total integrated cost =  5.898276690281298
step size too small or too large
RUN  10 , total integrated cost =  5.798205410953527
step size too small or too large
RUN  11 , total integrated cost =  5.922355489663713
RUN  12 , total integrated cost =  5.860845586835468
RUN  13 , total integrated cost =  5.77139873188206
RUN  14 , total integrated cost =  5.857038599602771
RUN  15 , total integrated cost =  5.560427858761058
RUN  16 , total integrated cost =  5.704432771978311
step size too small or too large
RUN  17 , total integrated cost =  5.622551165626381
RUN  18 , total integrated cost =  5.383307856528139
step size too small or too large
step size too small or too large
RUN  19 , total integrated cost =  5.45128594368108
step size too small or too large
RUN  20 , total integrated cost =  

step size too small or too large
RUN  9 , total integrated cost =  5.908529878725365
RUN  10 , total integrated cost =  5.842736154829681
RUN  11 , total integrated cost =  5.710957292624723
step size too small or too large
RUN  12 , total integrated cost =  5.738832132749196
step size too small or too large
step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  5.768711977983505
RUN  14 , total integrated cost =  5.807439295941327
step size too small or too large
RUN  15 , total integrated cost =  5.657139485795
RUN  16 , total integrated cost =  5.636092351003745
step size too small or too large
step size too small or too large
RUN  17 , total integrated cost =  5.533003685455
step size too small or too large
RUN  18 , total integrated cost =  5.460551580059922
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  19 , total integrated cost =  5.45153437612

step size too small or too large
step size too small or too large
RUN  20 , total integrated cost =  5.868941809564568
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  21 , total integrated cost mean =  5.766343576784382
Improved over  21  iterations in  120.9309029559954  seconds by  30.409848639301927  percent.
cost =  6.280937894572203
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  521.7654365426832
Gradient descend method:  None
RUN  1 , total integrated cost =  48.265186084212516
RUN  2 , total integrated cost =  28.823271655527762
RUN  3 , total integrated cost =  7.52233009532072
RUN  3 , total integrated cost mean =  7.52233009532072
Improved over  3  iterations in  8.578379191050772  seconds by  98.55829275600064  percent.
interpolate adjoint :  True True True
RUN  0 ,

In [22]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 2
step_method_ind = 0

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
                
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
n =  2
cost =  6.847291243414146
iteration =  6
cost =  0.0
iteration =  12
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3671.419530522828
Gradient descend method:  None
RUN  1 , total integrated cost =  157.97532921640914
RUN  2 , total integrated cost =  116.74358730722034
RUN  3 , total integrated cost =  60.7874760865377
RUN  3 , total integrated cost mean =  60.7874760865377
Improved over  3  iterations in  0.385289358033333  seconds by  98.34430591270834  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  62.25829332238937
Gradient descend method:  None
RUN  1 , total integrated cost =  15.31616180404004
RUN  2 , total integrated cost =  11.335102445748444
RUN  3 , total integrated cost =  10.725320979059322
RUN  4 , total integrated cost =  10.542590119794168
RUN  5 , total integrated cost =  10.404612767393779
RUN  6 , total integrated cost = 

step size too small or too large
RUN  1 , total integrated cost =  6.233785036998936
RUN  2 , total integrated cost =  5.190593949866109
RUN  3 , total integrated cost =  5.11065408445219
RUN  4 , total integrated cost =  5.025010729231113
step size too small or too large
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  4.755125600717352
step size too small or too large
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  4.413662292713976
RUN  7 , total integrated cost =  4.276418153020814
step size too small or too large
RUN  8 , total integrated cost =  4.170034350044666
step size too small or too large
RUN  9 , total integrated cost =  4.147757789051479
RUN  9 , total integrated cost mean =  4.147757789051479
Improved over  9  iterations in  11.32688413496362  seconds by  54.33210065665866  percent.
rep_ind =  30.177845936037365
cost =  6.604402815147046
iteration =  18
cost =  0.0
ite

RUN  0 , total integrated cost =  3681.8429328905745
Gradient descend method:  None
RUN  1 , total integrated cost =  155.2481207061651
RUN  2 , total integrated cost =  10.717708962639783
RUN  3 , total integrated cost =  9.398602472264363
RUN  3 , total integrated cost mean =  9.398602472264363
Improved over  3  iterations in  5.118403328990098  seconds by  99.7447310316172  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  8.810185305176573
Gradient descend method:  None
RUN  1 , total integrated cost =  6.872273386049579
step size too small or too large
RUN  2 , total integrated cost =  5.485781866966002
RUN  3 , total integrated cost =  5.001682631287553
step size too small or too large
RUN  4 , total integrated cost =  4.324611401576507
RUN  5 , total integrated cost =  4.101502491028481
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  4.001573054324293
RUN  7 , total integrated cost =  3.9281248908

step size too small or too large
step size too small or too large
step size too small or too large
RUN  9 , total integrated cost =  4.634254383176394
RUN  9 , total integrated cost mean =  4.634254383176394
Improved over  9  iterations in  55.7926889579976  seconds by  77.61160551188232  percent.
rep_ind =  2.695803931053713
cost =  3.844146035587097
iteration =  18
cost =  0.0
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3681.429017950654
Gradient descend method:  None
RUN  1 , total integrated cost =  145.71112468639984
RUN  2 , total integrated cost =  32.49142644250884
RUN  3 , total integrated cost =  9.120875205667645
RUN  3 , total integrated cost mean =  9.120875205667645
Improved over  3  iterations in  9.38738241797546  seconds by  99.7522463380064  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  9.039041803142128
Gradient descend method:  None
RUN  1 , total integrated cost =  7.212658400419614
step

RUN  1 , total integrated cost =  10.193086539842998
RUN  2 , total integrated cost =  9.26515279385978
RUN  3 , total integrated cost =  8.756646686849212
RUN  4 , total integrated cost =  8.86237948170801
RUN  5 , total integrated cost =  8.694209209136593
RUN  6 , total integrated cost =  8.616784883723913
RUN  7 , total integrated cost =  8.872299059060708
RUN  8 , total integrated cost =  8.392758736173104
RUN  9 , total integrated cost =  8.609393179182495
RUN  9 , total integrated cost mean =  8.609393179182495
Improved over  9  iterations in  1.8755463999696076  seconds by  57.479167532462036  percent.
rep_ind =  8.403551035677982
cost =  7.9792319701713685
iteration =  18
cost =  0.0
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1222.1670973127796
Gradient descend method:  None
RUN  1 , total integrated cost =  81.74190211713844
RUN  2 , total integrated cost =  28.59756187955551
RUN  3 , total integrated cost =  10.338615536976665
RUN

...

RUN  12 , total integrated cost =  7.790038290474337
RUN  13 , total integrated cost =  7.72178439079535
step size too small or too large
step size too small or too large
RUN  14 , total integrated cost =  7.401453444882213
step size too small or too large
step size too small or too large
step size too small or too large
RUN  15 , total integrated cost =  7.239400111268693
step size too small or too large
step size too small or too large
RUN  16 , total integrated cost =  7.149794276470028
RUN  17 , total integrated cost =  6.71834235129083
RUN  18 , total integrated cost =  6.637077946191237
step size too small or too large
step size too small or too large
RUN  19 , total integrated cost =  6.2687856426632385
RUN  20 , total integrated cost =  5.52410496837637
RUN  21 , total integrated cost mean =  5.08423916410126
Improved over  21  iterations in  64.10721099900547  seconds by  56.05744287240338  percent.
rep_ind =  2.7043480721277997
n =  50
cost =  7.492934575130731
iteration =  6

RUN  3 , total integrated cost =  10.694242077905884
RUN  3 , total integrated cost mean =  10.694242077905884
Improved over  3  iterations in  8.325016785005573  seconds by  98.53868270026386  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  10.913168722759437
Gradient descend method:  None
RUN  1 , total integrated cost =  9.851665115478237
step size too small or too large
RUN  2 , total integrated cost =  9.525159821075786
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  9.251953969264646
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  9.006537209831258
RUN  5 , total integrated cost =  8.530679527809125
RUN  6 , total integrated cost =  8.349082273222509
step size too small or too large


RUN  0 , total integrated cost =  528.48127612246
Gradient descend method:  None
RUN  1 , total integrated cost =  30.66904886508801
RUN  2 , total integrated cost =  8.687415284657293
RUN  3 , total integrated cost =  5.947536742830451
RUN  3 , total integrated cost mean =  5.947536742830451
Improved over  3  iterations in  1.6354554640129209  seconds by  98.8745983989313  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  6.178728213996637
Gradient descend method:  None
RUN  1 , total integrated cost =  5.655464004047033
RUN  2 , total integrated cost =  5.567167113336801
step size too small or too large
RUN  3 , total integrated cost =  5.573348120606451
RUN  4 , total integrated cost =  6.022249864492944
RUN  5 , total integrated cost =  5.836612063380364
no cost improvement
RUN  6 , total integrated cost =  5.567004692596564
RUN  6 , total integrated cost mean =  5.567004692596564
Improved over  6  iterations in  6.263695447996724  seconds by  9.90047

RUN  1 , total integrated cost =  7.574186784681151
step size too small or too large
RUN  2 , total integrated cost =  7.415091461537881
RUN  3 , total integrated cost =  7.004262461459125
step size too small or too large
RUN  4 , total integrated cost =  6.936118770874802
RUN  5 , total integrated cost =  6.657169348916023
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  6.155924733439231
RUN  7 , total integrated cost =  6.387240126040321
step size too small or too large
RUN  8 , total integrated cost =  6.230963973078672
step size too small or too large
step size too small or too large
step size too small or too large
RUN  9 , total integrated cost =  6.0915035643254685
RUN  9 , total integrated cost mean =  6.0915035643254685
Improved over  9  iterations in  26.933885877020657  seconds by  26.01645823514052  percent.
rep_ind =  8.713116104028463
cost =  6.643703946029167
iteration =  18
cost =  0.0
iteration =  24
interpolate adjoi

step size too small or too large
RUN  3 , total integrated cost =  8.491602544383374
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  8.053664976209289
step size too small or too large
RUN  5 , total integrated cost =  7.790844400685212
step size too small or too large
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  7.226037673932092
step size too small or too large
RUN  7 , total integrated cost =  6.631820192483283
RUN  8 , total integrated cost =  6.067230637495252
step size too small or too large
RUN  9 , total integrated cost =  5.764871586141444
RUN  9 , total integrated cost mean =  5.764871586141444
Improved over  9  iterations in  45.46547515800921  seconds by  42.57894422258425  percent.
rep_ind =  3.847540199403

In [23]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 4
method_ind = 3
step_method_ind = 0
step_m = 'S1'

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if ( d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ):
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
                            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)

                
                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
                            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
                                

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.008817022660197538 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.049999999999999996
set cost params:  0.005282379365665093 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n = 

In [29]:
for s_ind in range(len(d['sigma'])):
    for n_ind in range(len(d['N_array'])):
        for i_it in range(1,len(d['it_array'])):
            #d['cost'][s_ind, 3, 0, n_ind, i_it] = 0.

In [24]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 2
step_method_ind = 1
step_m = 'S1'
if step_method_ind == 1:
    step_m = 'S2'

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
n =  2
iteration =  6
iteration =  12
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3670.5459710649557
Gradient descend method:  None
RUN  1 , total integrated cost =  165.6427619143517
RUN  2 , total integrated cost =  138.12580829189324
RUN  3 , total integrated cost =  68.26767758769049
RUN  3 , total integrated cost mean =  68.26767758769049
Improved over  3  iterations in  0.40914111700840294  seconds by  98.1401219838725  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  67.07675598213858
Gradient descend method:  None
RUN  1 , total integrated cost =  15.256401650984115
RUN  2 , total integrated cost =  11.643674298251842
RUN  3 , total integrated cost =  10.803286235229432
RUN  4 , total integrated cost =  10.74094940624164
RUN  5 , total integrated cost =  10.643876634233138
RUN  6 , total integrated cost =  10.33477296514653
RUN  7 , total i

RUN  1 , total integrated cost =  6.292627441517633
RUN  2 , total integrated cost =  6.077730649064388
RUN  3 , total integrated cost =  5.8565880563295085
step size too small or too large
RUN  4 , total integrated cost =  5.7707453052804905
step size too small or too large
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  5.723316236112643
RUN  6 , total integrated cost =  5.636111114230714
RUN  7 , total integrated cost =  5.589327230686396
RUN  8 , total integrated cost =  5.49251255635822
RUN  9 , total integrated cost =  5.460171376073022
RUN  9 , total integrated cost mean =  5.460171376073022
Improved over  9  iterations in  9.661612530006096  seconds by  32.13099642513609  percent.
rep_ind =  14.525402205281393
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3688.2964628924065
Gradient descend method:  None
RUN  1 , total integrated cost =  144.85664181613205
RUN  2 , tota

RUN  1 , total integrated cost =  7.836881648895458
step size too small or too large
RUN  2 , total integrated cost =  7.218179767818029
step size too small or too large
RUN  3 , total integrated cost =  6.844347776097098
RUN  4 , total integrated cost =  5.940477539713699
RUN  5 , total integrated cost =  5.597786284035296
RUN  6 , total integrated cost =  5.502184660287966
RUN  7 , total integrated cost =  5.33829325244365
step size too small or too large
RUN  8 , total integrated cost =  4.956389238260133
RUN  9 , total integrated cost =  5.063116003702141
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  10 , total integrated cost =  4.760229890653608
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  11 , total in

step size too small or too large
step size too small or too large
RUN  8 , total integrated cost =  4.007026730192759
step size too small or too large
step size too small or too large
RUN  9 , total integrated cost =  4.036484601814114
RUN  9 , total integrated cost mean =  4.036484601814114
Improved over  9  iterations in  55.027402371983044  seconds by  23.952052986361082  percent.
rep_ind =  8.225692123143439
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3680.4316256725865
Gradient descend method:  None
RUN  1 , total integrated cost =  149.89121781418527
RUN  2 , total integrated cost =  65.20066229694
RUN  3 , total integrated cost =  43.13887391777335
RUN  3 , total integrated cost mean =  43.13887391777335
Improved over  3  iterations in  8.570678355987184  seconds by  98.8278854681918  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  42.841499671270505
Gradient descend method:  None
RUN  1

RUN  1 , total integrated cost =  7.757735096678928
RUN  2 , total integrated cost =  7.7748803770086745
RUN  3 , total integrated cost =  7.677383502169377
RUN  4 , total integrated cost =  7.7361490667695065
RUN  5 , total integrated cost =  7.580448718253036
RUN  6 , total integrated cost =  7.701105591628753
RUN  7 , total integrated cost =  7.655736696472698
RUN  8 , total integrated cost =  7.613766220939651
RUN  9 , total integrated cost =  7.67325916740304
RUN  9 , total integrated cost mean =  7.67325916740304
Improved over  9  iterations in  1.7931155869737267  seconds by  12.707053739816004  percent.
rep_ind =  5.936300949087974
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1221.4034710574642
Gradient descend method:  None
RUN  1 , total integrated cost =  47.83628399433937
RUN  2 , total integrated cost =  12.619560500469117
RUN  3 , total integrated cost =  6.925255120509121
RUN  3 , total integrated cost mean =  6.

...

RUN  4 , total integrated cost =  8.110677424031984
RUN  5 , total integrated cost =  7.822300082088202
RUN  6 , total integrated cost =  7.1598899528709765
RUN  7 , total integrated cost =  6.351853059734782
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  8 , total integrated cost =  6.048576405551427
step size too small or too large
RUN  9 , total integrated cost =  5.935000903282177
RUN  9 , total integrated cost mean =  5.935000903282177
Improved over  9  iterations in  44.8482564050355  seconds by  37.97865550165815  percent.
cost =  6.115955072938465
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  733.6865765859569
Gradient descend method:  None
RUN  1 , total integrated cost =  40.37694059172727
RUN  2 , total integrated cost =  19.279607440157115
RUN  3 , total integrated cost =  10.521100449860205
RUN

RUN  18 , total integrated cost =  5.265036975368007
step size too small or too large
RUN  19 , total integrated cost =  4.983547552409743
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  20 , total integrated cost =  4.95937386690911
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  21 , total integrated cost mean =  4.918339624392486
Improved over  21  iterations in  108.75566697103204  seconds by  54.379952341691  percent.
rep_ind =  2.2673786134993863
sigma =  0.06999999999999999
set cost params:  0.003772423157889073 0.0 1.0
n =  2
iteration =  6
iteration =  12
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  526.8976921516125
Gradient descend method:  None
RUN  1 , total integrated cost =  66.92767483780483
RUN  2 , tot

step size too small or too large
RUN  1 , total integrated cost =  8.0825595300667
RUN  2 , total integrated cost =  7.900248555201105
RUN  3 , total integrated cost =  7.275882440263333
RUN  4 , total integrated cost =  7.156019549810066
RUN  5 , total integrated cost =  6.934211976315417
RUN  6 , total integrated cost =  7.021218577403498
step size too small or too large
RUN  7 , total integrated cost =  6.81986909893632
step size too small or too large
step size too small or too large
RUN  8 , total integrated cost =  6.3788148970909955
RUN  9 , total integrated cost =  6.339746666454472
RUN  9 , total integrated cost mean =  6.339746666454472
Improved over  9  iterations in  9.065197128977161  seconds by  26.88103484006875  percent.
rep_ind =  5.0543347901535105
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  525.044369645466
Gradient descend method:  None
RUN  1 , total integrated cost =  19.749888001547664
step size too smal

RUN  7 , total integrated cost =  5.810566423131035
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  8 , total integrated cost =  5.660586595281102
RUN  9 , total integrated cost =  5.435157505431836
RUN  10 , total integrated cost =  5.586312153195313
RUN  11 , total integrated cost =  5.736632570207245
RUN  12 , total integrated cost =  5.9649027302805875
step size too small or too large
step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  5.739547685011652
RUN  14 , total integrated cost =  5.754802557739565
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  15 , total integrated cost =  5.741174088939368
RUN  16 , total integrated cost =  5.642416657553197
step size too small or too large
step size 

RUN  3 , total integrated cost mean =  6.6751303134963385
Improved over  3  iterations in  7.692456846009009  seconds by  98.71922681692259  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  6.633983038635054
Gradient descend method:  None
step size too small or too large
RUN  1 , total integrated cost =  6.328307298465711
step size too small or too large
RUN  2 , total integrated cost =  5.906918201316711
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  5.681487170590336
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  5.820313132608008
RUN  5 , total integrated cost =  6.146980004016569
step size too small or too large
step size too small or too large
step size too small or too large
RUN  6 , total integrated cost =  5.866212636335267
step size too small or too large
step size too small or too large
step size too small o

In [33]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 3
step_method_ind = 1
step_m = 'S1'
if step_method_ind == 1:
    step_m = 'S2'

for d in dict_list:
    
    if d['point'][0] != 0.5:
        continue
    if d['point'][1] != 0.5:
        continue
        
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):

        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.5 0.5]
sigma =  0.01
set cost params:  0.026458600489613113 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.008817022660197538 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.049999999999999996
set cost params:  0.005282379365665093 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n = 